In [1]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

In [2]:
streets = gpd.read_file('../data/nyc_clean.geojson')
streets.head(3)

,name,boro,osm_name,front_card,middle_card,back_card,st_num,way_type,way_name,geometry
0,100 avenue,queens,100th avenue,None,None,None,100,avenue,None,"MULTILINESTRING ((-73.74175 40.71573, -73.7413..."
1,100 drive,queens,100th drive,None,None,None,100,drive,None,"MULTILINESTRING ((-73.73203 40.71458, -73.7319..."
2,100 place,queens,100th place,None,None,None,100,place,None,"LINESTRING (-73.81879 40.60576, -73.81889 40.6..."


In [3]:
streets.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9436 entries, 0 to 9435
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   name         9436 non-null   object  
 1   boro         9436 non-null   object  
 2   osm_name     9436 non-null   object  
 3   front_card   883 non-null    object  
 4   middle_card  9 non-null      object  
 5   back_card    150 non-null    object  
 6   st_num       2004 non-null   object  
 7   way_type     8998 non-null   object  
 8   way_name     7735 non-null   object  
 9   geometry     9436 non-null   geometry
dtypes: geometry(1), object(9)
memory usage: 737.3+ KB


In [4]:
# Check all streets for intersections
intersections = []
for i in range(0, len(streets)):
    row1 = streets.iloc[i]
    geo1 = row1['geometry']
    name1 = row1['name']
    boro1 = row1['boro']
    for j in range(i + 1, len(streets)):
        row2 = streets.iloc[j]
        geo2 = row2['geometry']
        if geo1.intersects(geo2):
            name2 = row2['name']
            boro2 = row2['boro']
            inter = geo1.intersection(geo2)
            inter_dict1 = {
                'street_1': name1,
                'street_2': name2,
                'boro_1': boro1,
                'boro_2': boro2,
                'geometry': inter
            }
            inter_dict2 = {
                'street_1': name2,
                'street_2': name1,
                'boro_1': boro2,
                'boro_2': boro1,
                'geometry': inter
            }
            intersections.append(inter_dict1)
            intersections.append(inter_dict2)


In [5]:
inter_geos = gpd.GeoDataFrame(intersections)

In [6]:
inter_geos.head()

,street_1,street_2,boro_1,boro_2,geometry
0,100 avenue,100 drive,queens,queens,POINT (-73.73408 40.71508)
1,100 drive,100 avenue,queens,queens,POINT (-73.73408 40.71508)
2,100 avenue,193 street,queens,queens,POINT (-73.76575 40.70904)
3,193 street,100 avenue,queens,queens,POINT (-73.76575 40.70904)
4,100 avenue,194 street,queens,queens,POINT (-73.76482 40.70926)


In [7]:
inter_geos.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 90246 entries, 0 to 90245
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   street_1  90246 non-null  object  
 1   street_2  90246 non-null  object  
 2   boro_1    90246 non-null  object  
 3   boro_2    90246 non-null  object  
 4   geometry  90246 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 3.4+ MB


In [8]:
inter_geos.to_file('../data/intersections.geojson', driver='GeoJSON')